# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from faststripe.endpoints import eps
from faststripe.spec import docs_url
from inspect import Parameter, Signature, signature
from urllib.parse import quote

import re,httpx

In [ ]:
#| hide
from nbdev import *

import itertools, os

In [ ]:
#| exports
stripe_api_url = 'https://api.stripe.com'

In [ ]:
ep = first(eps, lambda x: x['data'] and len(x['data']) < 3)
ep

{'data': [{'annotation': list,
   'default': None,
   'description': 'Specifies which fields in the response should be expanded.',
   'name': 'expand'},
  {'annotation': bool,
   'default': None,
   'description': "To request a new capability for an account, pass true. There can be a delay before the requested capability becomes active. If the capability has any activation requirements, the response includes them in the `requirements` arrays.\n\nIf a capability isn't permanent, you can remove it from the account by passing false. Some capabilities are permanent after they've been requested. Attempting to remove a permanent capability returns an error.",
   'name': 'requested'}],
 'doc_url': 'https://docs.stripe.com/api/capabilities/update',
 'op_id': 'PostAccountsAccountCapabilitiesCapability',
 'path': '/v1/accounts/{account}/capabilities/{capability}',
 'qparams': [],
 'summary': 'Update an Account Capability',
 'verb': 'post'}

## Parsing Endpoints

Now, with each of these descriptions, we can easily create a request that we want on the fly. However, to make it a little bit nicer to use in a library, we'll go ahead and automatically generate classes with proper signatures and docstrings that are then easily accessible in any standard IDE.

In [ ]:
#| export
def _mk_param(name, **kwargs):
    kwargs.pop('description', None)
    if name == 'expand': kwargs['annotation'] = list
    return Parameter(name, kind=Parameter.POSITIONAL_OR_KEYWORD, **kwargs)

def _mk_sig(req_args, opt_args, anno_args):
    'req_args are args inside paths such as {account}, opt_args are query params, anno_args are payload params'
    params =  [_mk_param(k) for k in req_args]
    params += [_mk_param(**{'default': None} | p) for p in opt_args or anno_args]
    return Signature(params)

In [ ]:
path, *_ = partial_format(ep['path'])
req_args = stringfmt_names(path)
opt_args = ep['qparams']
anno_args = ep['data']

_mk_sig(req_args, opt_args, anno_args)

<Signature (account, capability, expand: list = None, requested: bool = None)>

For organizing our APIs into resource groups, which the OpenAPI spec tends to do, we're going to parse the operation ID.

In [ ]:
#| export
def op2nm(op_id):
    'Parse the operation ID to get the resource and name'
    parts = re.findall(r'[A-Z][a-z]*', op_id)
    verb, *nm = [p.lower() for p in parts]
    res, nm = nm[0], nm[1:]
    nm += [verb]
    return res, nm

In [ ]:
op2nm(ep['op_id'])

('accounts', ['account', 'capabilities', 'capability', 'post'])

In [ ]:
#| export
class _OAPIObj: pass

In [ ]:
#| export
def _flatten_data(data, prefix=''):
    'Flatten a dictionary of data so that it can be used in a request body.'
    result = {}
    for k,v in data.items():
        key = f'{prefix}[{k}]' if prefix else k
        if isinstance(v, dict): result.update(_flatten_data(v, key))
        elif isinstance(v, list): 
            for i,item in enumerate(v):
                if isinstance(item, dict): result.update(_flatten_data(item, f'{key}[{i}]'))
                else: result[f'{key}[{i}]'] = item
        else: result[key] = v
    return result

In [ ]:
stripe_key = os.environ['STRIPE_SECRET_KEY']
headers = {'Authorization': f'Bearer {stripe_key}'}

In [ ]:
#| export
names = lambda x: [o['name'] for o in x]

class _OAPIVerb(_OAPIObj):
    __slots__ = 'path verb res name summary pparams qparams url data doc_url hdrs client __doc__'.split()
    def __init__(self, path, verb, op_id, summary, qparams, data, doc_url, url, hdrs, client, kwargs={}):
        res, name = op2nm(op_id) # custom per oapi spec
        name = '_'.join(name)
        path, *_ = partial_format(path, **kwargs)
        pparams = stringfmt_names(path) # params inside path
        param_docs = '\n'.join(f"    {p['name']}: {p['description']}" for p in qparams + data)
        __doc__ = f"{summary}\n\nParameters:\n{param_docs}" if param_docs else summary
        store_attr()

    def __call__(self, *args, **kwargs):
        'Call the API endpoint with the given arguments'
        flds = self.pparams + names(self.qparams) + names(self.data)
        for a, b in zip(args, flds): kwargs[b] = a
        route_p, query_p, data_p = [{p: kwargs[p] for p in o if p in kwargs}
                                    for o in (self.pparams, names(self.qparams), names(self.data))]
        return self.client(self.path, self.verb, route=route_p, query=query_p, data=data_p)

    def __str__(self): return f'{self.res}.{self.name}{signature(self)}'
    @property
    def __signature__(self): return _mk_sig(self.pparams, self.qparams, self.data)
    __call__.__signature__ = __signature__

    def _repr_markdown_(self):
        return f'[{self.res}.{self.name}]({self.doc_url}){self.__signature__}: *{self.summary}*'
    __repr__ = _repr_markdown_

With this, we can now construct a entire OpenAPI verb with proper documentation and nice repr inside Jupyter environments!

In [ ]:
v = _OAPIVerb(url=stripe_api_url, hdrs=headers, client=None, **eps[0])
v

[account.get](https://docs.stripe.com/api/accounts/retrieve)(expand: list = None): *Retrieve account*

Each endpoint includes a direct link to its documentation page. Due to Stripe's API organization, some links (like checkout endpoints) may return 404s, but most core endpoints link correctly to their documentation when you need additional context.

In [ ]:
v?

```python
def get(expand: list = None)(
    expand:list=None
):

```



```
Retrieve account

Parameters:
    expand: Specifies which fields in the response should be expanded.
```



**Type:** _OAPIVerb

Let's go ahead and group our verbs based on the resource they are apart of. We can setup a repr to make discoverability of operations you can perform easy.

In [ ]:
#| export
class _OAPIVerbGroup(_OAPIObj):
    def __init__(self, name, verbs):
        self.name,self.verbs = name,verbs
        for o in verbs: setattr(self, o.name, o)
    def __str__(self): return "\n".join(str(v) for v in self.verbs)
    def _repr_markdown_(self): return "\n".join(f'- {v._repr_markdown_()}' for v in self.verbs)

In [ ]:
verbs = L(eps).map(lambda x: _OAPIVerb(**x, url=stripe_api_url, hdrs=headers, client=None))
groups = {k.replace('-','_'): _OAPIVerbGroup(k,v) for k,v in groupby(verbs, 'res').items()}
res, g = first(groups.items())
g

- [account.get](https://docs.stripe.com/api/accounts/retrieve)(expand: list = None): *Retrieve account*
- [account.links_post](https://docs.stripe.com/api/account_links/create)(account: str = None, collect: str = None, collection_options: dict = None, expand: list = None, refresh_url: str = None, return_url: str = None, type: str = None): *Create an account link*
- [account.sessions_post](https://docs.stripe.com/api/account_sessions/create)(account: str = None, components: dict = None, expand: list = None): *Create an Account Session*

## StripeApi

Let's go ahead and design a class since we need to store our API key and for use in the headers for each of these classes.

In [ ]:
#| export
class StripeApi:
    def __init__(self, api_key=None, base_url=stripe_api_url):
        if not api_key: api_key = os.getenv('STRIPE_SECRET_KEY', os.getenv('STRIPE_API_KEY'))
        self.api_key,self.base_url = api_key,base_url.strip('/')
        self.hdrs = {'Authorization': f'Bearer {self.api_key}'}
        verbs = L(eps).map(lambda x: _OAPIVerb(**x, url=base_url, hdrs=self.hdrs, client=self))
        self.func_dict = {f'{o.path}:{o.verb.upper()}':o for o in verbs}
        self.groups = {k.replace('-','_'): _OAPIVerbGroup(k,v) for k,v in groupby(verbs, 'res').items()}
    
    def _prep(self, path, verb, headers, route, query, data):
        headers = {**self.hdrs, **(headers or {})}
        path = path.strip('/')
        if route: path = path.format(**{k: quote(str(v)) for k,v in route.items()})
        if query: query = _flatten_data(query)
        if data: data = _flatten_data(data)
        return path, verb or 'GET', headers, query, data

    def __call__(self, path:str, verb:str=None, headers:dict=None, route:dict=None, query:dict=None, data=None):
        'Call the API endpoint with the given arguments'
        path, verb, headers, query, data = self._prep(path, verb, headers, route, query, data)
        resp = httpx.request(verb, f"{self.base_url}/{path}", headers=headers, params=query, data=data).raise_for_status()
        self.recv_hdrs = dict(resp.headers)
        return dict2obj(resp.json())
    
    def __dir__(self): return super().__dir__() + list(self.groups)
    def _repr_markdown_(self): return "\n".join(f"- [{o}]({docs_url + '/' + o})" for o in sorted(self.groups))
    def __getattr__(self,k): return self.groups[k] if 'groups' in vars(self) and k in self.groups else stop(AttributeError(k))
    def __getitem__(self, k):
        "Lookup and call an endpoint by path and verb (which defaults to 'GET')"
        a,b = k if isinstance(k,tuple) else (k,'GET')
        return self.func_dict[f'{a}:{b.upper()}']

In [ ]:
#| export
class StripeApiAsync(StripeApi):
    async def __call__(self, path:str, verb:str=None, headers:dict=None, route:dict=None, query:dict=None, data=None):
        'Call the API endpoint with the given arguments'
        path, verb, headers, query, data = self._prep(path, verb, headers, route, query, data)
        async with httpx.AsyncClient() as cli:
            resp = (await cli.request(verb, f"{self.base_url}/{path}", headers=headers, params=query, data=data)).raise_for_status()
        self.recv_hdrs = dict(resp.headers)
        return dict2obj(resp.json())

In [ ]:
sapi = StripeApi() # Uses `STRIPE_SECRET_KEY` env var by default
sapi

- [account](https://docs.stripe.com/api/account)
- [accounts](https://docs.stripe.com/api/accounts)
- [apple](https://docs.stripe.com/api/apple)
- [application](https://docs.stripe.com/api/application)
- [apps](https://docs.stripe.com/api/apps)
- [balance](https://docs.stripe.com/api/balance)
- [billing](https://docs.stripe.com/api/billing)
- [charges](https://docs.stripe.com/api/charges)
- [checkout](https://docs.stripe.com/api/checkout)
- [climate](https://docs.stripe.com/api/climate)
- [confirmation](https://docs.stripe.com/api/confirmation)
- [country](https://docs.stripe.com/api/country)
- [coupons](https://docs.stripe.com/api/coupons)
- [credit](https://docs.stripe.com/api/credit)
- [customer](https://docs.stripe.com/api/customer)
- [customers](https://docs.stripe.com/api/customers)
- [disputes](https://docs.stripe.com/api/disputes)
- [entitlements](https://docs.stripe.com/api/entitlements)
- [ephemeral](https://docs.stripe.com/api/ephemeral)
- [events](https://docs.stripe.com/api/events)
- [exchange](https://docs.stripe.com/api/exchange)
- [external](https://docs.stripe.com/api/external)
- [fabric](https://docs.stripe.com/api/fabric)
- [file](https://docs.stripe.com/api/file)
- [files](https://docs.stripe.com/api/files)
- [financial](https://docs.stripe.com/api/financial)
- [forwarding](https://docs.stripe.com/api/forwarding)
- [identity](https://docs.stripe.com/api/identity)
- [invoice](https://docs.stripe.com/api/invoice)
- [invoiceitems](https://docs.stripe.com/api/invoiceitems)
- [invoices](https://docs.stripe.com/api/invoices)
- [issuing](https://docs.stripe.com/api/issuing)
- [link](https://docs.stripe.com/api/link)
- [linked](https://docs.stripe.com/api/linked)
- [mandates](https://docs.stripe.com/api/mandates)
- [payment](https://docs.stripe.com/api/payment)
- [payouts](https://docs.stripe.com/api/payouts)
- [plans](https://docs.stripe.com/api/plans)
- [prices](https://docs.stripe.com/api/prices)
- [products](https://docs.stripe.com/api/products)
- [promotion](https://docs.stripe.com/api/promotion)
- [quotes](https://docs.stripe.com/api/quotes)
- [radar](https://docs.stripe.com/api/radar)
- [refunds](https://docs.stripe.com/api/refunds)
- [reporting](https://docs.stripe.com/api/reporting)
- [reviews](https://docs.stripe.com/api/reviews)
- [setup](https://docs.stripe.com/api/setup)
- [shipping](https://docs.stripe.com/api/shipping)
- [sigma](https://docs.stripe.com/api/sigma)
- [sources](https://docs.stripe.com/api/sources)
- [subscription](https://docs.stripe.com/api/subscription)
- [subscriptions](https://docs.stripe.com/api/subscriptions)
- [tax](https://docs.stripe.com/api/tax)
- [terminal](https://docs.stripe.com/api/terminal)
- [test](https://docs.stripe.com/api/test)
- [tokens](https://docs.stripe.com/api/tokens)
- [topups](https://docs.stripe.com/api/topups)
- [transfers](https://docs.stripe.com/api/transfers)
- [treasury](https://docs.stripe.com/api/treasury)
- [webhook](https://docs.stripe.com/api/webhook)

In [ ]:
sapi.account

- [account.get](https://docs.stripe.com/api/accounts/retrieve)(expand: list = None): *Retrieve account*
- [account.links_post](https://docs.stripe.com/api/account_links/create)(account: str = None, collect: str = None, collection_options: dict = None, expand: list = None, refresh_url: str = None, return_url: str = None, type: str = None): *Create an account link*
- [account.sessions_post](https://docs.stripe.com/api/account_sessions/create)(account: str = None, components: dict = None, expand: list = None): *Create an Account Session*

In [ ]:
sapi.account.get().keys()

dict_keys(['id', 'object', 'business_profile', 'business_type', 'capabilities', 'charges_enabled', 'company', 'controller', 'country', 'default_currency', 'details_submitted', 'email', 'payouts_enabled', 'settings', 'type'])

You can also call `StripeApi` directly with the path, verb, parameters, headers, etc:

In [ ]:
prod = sapi('/v1/products', 'POST', data=dict(name='Test Product'))
prod.id, prod.name

('prod_Tx1xp1BieLA0US', 'Test Product')

In [ ]:
show_doc(StripeApi.__getitem__)

---

[source](https://github.com/AnswerDotAI/faststripe/blob/main/faststripe/core.py#L124){target="_blank" style="float:right; font-size:smaller"}

### StripeApi.__getitem__

```python

def __getitem__(
    k
):


```

*Lookup and call an endpoint by path and verb (which defaults to 'GET')*

You can access endpoints by indexing into the object. When using the API this way, you do not need to specify what type of parameter (route, query, or post data) is being used. This is, therefore, the same call as above:

In [ ]:
prod = sapi['/v1/products'](name='Test Product') # defaults to GET
prod.data[0].id, prod.data[0].name

('prod_Tx1xp1BieLA0US', 'Test Product')

That is all we need in order to have a fully functional Python SDK that is compliant with the Stripe OpenAPI spec. Kind of insane that in under 100 lines of code, we can get this functionality, which in my opinion is in some respects even better than the official Stripe Python SDK for the simple fact that we can see the parameters that the functions take without looking up the API reference doc online.

Let's go ahead and try to build with this thing. The simplest payment system that you can have in Stripe is a one-time payment URL. Here is how we do this in our new API. First, we have to create a product and its price.

In [ ]:
prod = sapi.products.post(name='Test Product')
prod.id, prod.name

('prod_Tx1xOoz9nZNsJP', 'Test Product')

In [ ]:
price = sapi.prices.post(product=prod.id, unit_amount=10_00, currency='usd')
price.id, price.unit_amount, price.currency

('price_1Sz7ttKGhqIw9PXm6FYt5b9b', 1000, 'usd')

Use the async client to get async calls.

In [ ]:
asapi = StripeApiAsync()

In [ ]:
aprod = await asapi.products.post(name='Test Product')
aprod.id, aprod.name

('prod_Tx1xtPKLT157HW', 'Test Product')

Now we can create our checkout session with a mode of payment which means that it will only happen once and is not part of any sort of subscription.

In [ ]:
sapi.checkout

- [checkout.sessions_get](https://docs.stripe.com/api/sessions/list)(created: 'str' = None, customer: 'str' = None, customer_details: 'str' = None, ending_before: 'str' = None, expand: list = None, limit: 'str' = None, payment_intent: 'str' = None, payment_link: 'str' = None, starting_after: 'str' = None, status: 'str' = None, subscription: 'str' = None): *List all Checkout Sessions*
- [checkout.sessions_post](https://docs.stripe.com/api/sessions/create)(adaptive_pricing: dict = None, after_expiration: dict = None, allow_promotion_codes: bool = None, automatic_tax: dict = None, billing_address_collection: str = None, branding_settings: dict = None, cancel_url: str = None, client_reference_id: str = None, consent_collection: dict = None, currency: str = None, custom_fields: list = None, custom_text: dict = None, customer: str = None, customer_creation: str = None, customer_email: str = None, customer_update: dict = None, discounts: list = None, excluded_payment_method_types: list = None, expand: list = None, expires_at: int = None, invoice_creation: dict = None, line_items: list = None, locale: str = None, metadata: dict = None, mode: str = None, name_collection: dict = None, optional_items: list = None, origin_context: str = None, payment_intent_data: dict = None, payment_method_collection: str = None, payment_method_configuration: str = None, payment_method_data: dict = None, payment_method_options: dict = None, payment_method_types: list = None, permissions: dict = None, phone_number_collection: dict = None, redirect_on_completion: str = None, return_url: str = None, saved_payment_method_options: dict = None, setup_intent_data: dict = None, shipping_address_collection: dict = None, shipping_options: list = None, submit_type: str = None, subscription_data: dict = None, success_url: str = None, tax_id_collection: dict = None, ui_mode: str = None, wallet_options: dict = None): *Create a Checkout Session*
- [checkout.sessions_session_get](https://docs.stripe.com/api/sessions/delete)(session, expand: list = None): *Retrieve a Checkout Session*
- [checkout.sessions_session_post](https://docs.stripe.com/api/sessions/update)(session, collected_information: dict = None, expand: list = None, metadata: object = None, shipping_options: object = None): *Update a Checkout Session*
- [checkout.sessions_session_expire_post](https://docs.stripe.com/api/expires/update)(session, expand: list = None): *Expire a Checkout Session*
- [checkout.sessions_session_line_items_get](https://docs.stripe.com/api/line_items/delete)(session, ending_before: 'str' = None, expand: list = None, limit: 'str' = None, starting_after: 'str' = None): *Retrieve a Checkout Session's line items*

In [ ]:
checkout = sapi.checkout.sessions_post(mode='payment', line_items=[dict(price=price.id, quantity=1)],
                                       success_url='https://localhost:5001/success', cancel_url='https://localhost:5001/cancel')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1Yg2hX9DHet8IhchVmvU1KG...


Let's make this process even easier by add a higher level api ontop of our StripeApi.

## Pagination

First, let's make it a little bit easier to iterate through existing objects like customers or products.

The `paged` function provides an iterator that automatically handles pagination for Stripe API list methods.

In [ ]:
#| export
def paged(oper, *args, **kwargs):
    """Iterate through all pages of a Stripe API operation."""
    page = oper(*args, **kwargs)
    yield page
    while page.has_more:
        last_id = page.data[-1].id
        page = oper(*args, starting_after=last_id, **kwargs)
        yield page

In [ ]:
ps = L(itertools.islice(paged(sapi.customers.get, limit=2), 2))
cs = L(c for p in ps for c in p.data)
test_eq(len(ps), 2)
test_eq(len(cs), 4)

The `pages` function retrieves all items from all pages and returns them as a single list.

In [ ]:
#| export
def pages(oper, *args, **kwargs):
    """Retrieve all items from all pages of a Stripe API operation."""
    return L(p for page in paged(oper, *args, **kwargs) for p in page.data)

Let's test the pagination with the `customers` endpoint.

In [ ]:
customers = pages(sapi.customers.get, limit=100)
len(customers), customers[0].keys()

(1341,
 dict_keys(['id', 'object', 'address', 'balance', 'created', 'currency', 'customer_account', 'default_source', 'delinquent', 'description', 'discount', 'email', 'invoice_prefix', 'invoice_settings', 'livemode', 'metadata', 'name', 'next_invoice_sequence', 'phone', 'preferred_locales', 'shipping', 'tax_exempt', 'test_clock']))

## Helpers

Great, now let's make it easier to find products and prices!

In [ ]:
#| export
@patch
@delegates(StripeApi().products.get)
def find_product(self:StripeApi, name: str, limit=100, **kwargs):
    'Find a product by name'
    prods = pages(self.products.get, limit=limit, **kwargs)
    return first(prods, lambda p: p.name == name)

In [ ]:
sapi.find_product('Test Product')

```python
{ 'active': True,
  'attributes': [],
  'created': 1770055285,
  'default_price': {},
  'description': {},
  'id': 'prod_TuFx0aa85l7sIr',
  'images': [],
  'livemode': False,
  'marketing_features': [],
  'metadata': {},
  'name': 'Test Product',
  'object': 'product',
  'package_dimensions': {},
  'shippable': {},
  'statement_descriptor': {},
  'tax_code': {},
  'type': 'service',
  'unit_label': {},
  'updated': 1770055285,
  'url': {}}
```

In [ ]:
#| export
@patch
@delegates(StripeApi().prices.get)
def find_prices(self:StripeApi, product_id: str, limit=100, **kwargs):
    'Find all prices associated with a product id'
    return pages(self.prices.get, limit=limit, **kwargs).filter(lambda p: p.product == product_id)

In [ ]:
sapi.find_prices(sapi.find_product('Test Product').id)

[{'id': 'price_1SwRRBKGhqIw9PXmcbRkGznA', 'object': 'price', 'active': True, 'billing_scheme': 'per_unit', 'created': 1770055285, 'currency': 'usd', 'custom_unit_amount': {}, 'livemode': False, 'lookup_key': {}, 'metadata': {}, 'nickname': {}, 'product': 'prod_TuFx0aa85l7sIr', 'recurring': {}, 'tax_behavior': 'unspecified', 'tiers_mode': {}, 'transform_quantity': {}, 'type': 'one_time', 'unit_amount': 1000, 'unit_amount_decimal': '1000'}]

In [ ]:
#| export
@patch
def priced_product(self:StripeApi, product_name, amount_cents, currency='usd', recurring=None, description=None):
    "Create a product and price if they don't exist"
    prod_params = dict(name=product_name)
    if description: prod_params['description'] = description
    prod = self.find_product(product_name) or self.products.post(**prod_params)
    price_params = dict(product=prod.id, unit_amount=amount_cents, currency=currency)
    if recurring: price_params['recurring'] = recurring
    price = first(self.find_prices(prod.id)) or self.prices.post(**price_params)
    return prod, price

In [ ]:
prod, price = sapi.priced_product('Test Product', 10_00, 'usd')
prod.name, price.id, price.unit_amount

('Test Product', 'price_1SwRRBKGhqIw9PXmcbRkGznA', 1000)

Now we can automatically create a product or use an existing one when create a one time payment link.

In [ ]:
#| export
@patch
@delegates(StripeApi().checkout.sessions_post)
def one_time_payment(self:StripeApi, product_name, amount_cents, success_url, cancel_url, currency='usd', quantity=1, **kwargs):
    'Create a simple one-time payment checkout'
    automatic_tax = kwargs.pop('automatic_tax', {'enabled': True})
    _, price = self.priced_product(product_name, amount_cents, currency)
    return self.checkout.sessions_post(mode='payment', line_items=[dict(price=price.id, quantity=quantity)],
                                       automatic_tax=automatic_tax, success_url=success_url, cancel_url=cancel_url, **kwargs)

In [ ]:
checkout = sapi.one_time_payment('Test Product', 10_00, 'https://localhost:5001/success', 'https://localhost:5001/cancel', 'usd')
print(f'Payment link: {checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1K2vLpP2I89Df5WdJ8Ks9p8...


Another common use case is subscriptions. Let's go ahead and create a version that makes subscription creation just as easy.

In [ ]:
#| export
@patch
@delegates(StripeApi().checkout.sessions_post)
def subscription(self:StripeApi, product_name, amount_cents, success_url, cancel_url,
                 currency='usd', interval='month', **kwargs):
    'Create a simple recurring subscription'
    _, price = self.priced_product(product_name, amount_cents, currency, recurring=dict(interval=interval))
    return self.checkout.sessions_post(mode='subscription', success_url=success_url, cancel_url=cancel_url,
                                       line_items=[dict(price=price.id, quantity=1)], **kwargs)

In [ ]:
sub_checkout = sapi.subscription('Test Subscription Product', 10_00, 'https://localhost:5001/success', 'https://localhost:5001/cancel')
print(f'Payment link: {sub_checkout.url[:64]}...')

Payment link: https://billing.answer.ai/c/pay/cs_test_a1pchuEPwFRZJE2yeH84ciAJ...


**Note:** You'll want to use Stripe's webhook functionality for detecting payment and subscription events. To do so, you'll utilize the Python Stripe SDK that the event actually came from stripe.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()